<a href="https://colab.research.google.com/github/Birkbeck/bsc-computer-science-project-2021_22-mohammadreza490/blob/main/project_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
'''

1. Convert stems to wav. https://pypi.org/project/musdb/ 
2. For each song, split into 2 second segments. Add padding as needed
1. For each segment, create spectogram of whole mix of each song in training set via STFT
2. Create array containing spectograms of bass of each song in training set
3. Create array containing spectograms of (whole mix - bass) of each song in training set, see https://stackoverflow.com/questions/4039158/mixing-two-audio-files-together-with-python
4. Add padding to arrays such that shape is appropriate for the network.
4. Create CNN model
5. Train model
6. Apply inverse STFT / Get signal from the spectogram, see https://stackoverflow.com/questions/60377585/how-can-i-reverse-a-scipy-signal-spectrogram-to-audio-with-python
                                                        https://stackoverflow.com/questions/76447360/how-do-i-can-reconstructing-stft-to-audio
                                                        https://stackoverflow.com/questions/69387104/how-to-convert-wav-audio-file-from-mel-spectrogram
7. Test model with a bunch of songs

https://medium.com/@shameerayaseen21/u-net-advancing-image-segmentation-with-convolutional-neural-networks-1fd810f05d00
'''

'\n\n1. Convert stems to wav. https://pypi.org/project/musdb/ \n2. For each song, split into 2 second segments. Add padding as needed\n1. For each segment, create spectogram of whole mix of each song in training set via STFT\n2. Create array containing spectograms of bass of each song in training set\n3. Create array containing spectograms of (whole mix - bass) of each song in training set, see https://stackoverflow.com/questions/4039158/mixing-two-audio-files-together-with-python\n4. Add padding to arrays such that shape is appropriate for the network.\n4. Create CNN model\n5. Train model\n6. Apply inverse STFT / Get signal from the spectogram, see https://stackoverflow.com/questions/60377585/how-can-i-reverse-a-scipy-signal-spectrogram-to-audio-with-python\n                                                        https://stackoverflow.com/questions/76447360/how-do-i-can-reconstructing-stft-to-audio\n                                                        https://stackoverflow.com/ques

In [1]:
PATH_TO_SRC_FOLDER = r".\src"

In [2]:
%load_ext autoreload
%autoreload 2

In [14]:
!pip install musdb
!pip install numpy
!pip install pydub
!pip install "tensorflow<2.11" #!pip install tensorflow==2.14
# !pip install tensorflow-addons==0.22.0
!pip install ipdb
!pip install museval
!pip install librosa
!pip install matplotlib


ERROR: Invalid requirement: '#!pip'


In [3]:
import os
import sys

import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import IPython.display as ipd
import musdb
import subprocess
import pydub
# import tensorflow_addons as tfa
import time
import shutil
import ipdb
import scipy
import soundfile as sf
import museval
import random
import math

from scipy.io.wavfile import write
from glob import glob

In [4]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

print(tf.config.list_physical_devices('GPU'))

tf.Tensor(-753.5408, shape=(), dtype=float32)
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
if PATH_TO_SRC_FOLDER not in sys.path:
  sys.path.append(PATH_TO_SRC_FOLDER) #https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab

In [6]:
from wav_handler import WavHandler
from dataset_handler import DatasetHandler
from plotter import Plotter
from model_builder import ModelBuilder

In [7]:
plotter = Plotter()
model_builder = ModelBuilder()
dataset_handler = DatasetHandler()

In [8]:
unet = model_builder.buildModel()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 176, 1040,   0           []                               
                                1)]                                                               
                                                                                                  
 lambda (Lambda)                (None, 176, 1040, 1  0           ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 176, 1040, 3  320         ['lambda[0][0]']                 
                                2)                                                            

In [36]:
dataset_handler.writeTrainInputSpec()
dataset_handler.writeTrainOutputSpec()

'.\dataset\wav\train\A Classic Education - NightOwl\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Actions - Devil_s Words\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Actions - One Minute Smile\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Actions - South Of The Water\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Aimee Norwich - Child\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Alexander Ross - Goodbye Bolero\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Alexander Ross - Velvet Curtain\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Angela Thomas Wade - Milk Cow Blues\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\ANiMAL - Clinic A\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\ANiMAL - Easy Tiger\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\ANiMAL - Rockshow\padded_spectrograms.npy' already exists!
'.\dataset\wav\train\Atlantis

In [25]:
# DON'T RUN
# dataset_handler.createBassless()

trainInputSpecs = dataset_handler.loadTrainInputSpec()
trainOutputSpecs = dataset_handler.loadTrainOutputSpec()
# testSpecs = dataset_handler.loadTestSpec()

In [29]:
# print(len(glob(r".\dataset\wav\train\*\bassless_padded_spectrograms.npy")))

INDEX = 2

print(len(trainInputSpecs))
print(trainInputSpecs[INDEX].shape)

print(len(trainOutputSpecs))
print(trainOutputSpecs[INDEX].shape)

100
(99, 1040, 176)
100
(2, 99, 1040, 176)


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('bass_separator.h5', verbose=1, save_best_only=True)

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss')] # tf.keras.callbacks.TensorBoard(log_dir='logs')

results = unet.fit(trainInputSpecs, trainOutputSpecs, validation_split=0.1, batch_size=16, epochs=25, callbacks=callbacks)

In [20]:
wav_handler = WavHandler(r"dataset\wav\test\AM Contra - Heart Peripheral\mixture.wav")

segments = wav_handler.segmentWav()
spectrograms = wav_handler.computeSTFT(segments)
padded_spectrograms = wav_handler.zeroPadSTFT(spectrograms)

# istft = wav_handler.computeInverseSTFT(padded_spectrograms)

# write("istft.wav", rate = wav_handler.sr, data = istft.wav)

In [ ]:
firstInput = trainInputSpecs[0]
firstOutputBass, firstOutputBassless = trainOutputSpecs[0]
print(firstInput.shape)
print(firstOutputBass.shape)
print(firstOutputBassless.shape)

plotter.plotSpec(librosa.amplitude_to_db(firstInput[50]))